# Imports

In [1]:
import requests
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import re
from datetime import datetime
import pandas as pd
from connection_keys import connection, alchemy_connection

### Setup para abrir o navegador com o Selenium e puxar a primeira página

In [2]:
#--| Setup
options = Options()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
options.add_argument('--disable-extensions')
#options.add_argument("--headless")
browser = webdriver.Chrome(executable_path=r'chromedriver.exe', options=options)

In [3]:

--| Parse or automation
browser.get('https://www.kabum.com.br/hardware/placa-de-video-vga?pagina=1')
html = soup(browser.page_source, 'html.parser')
# browser.implicitly_wait(5)

### Configuiração para percorer as páginas e salvar todos dados com a taga referente ao produto em uma lista

In [4]:
pagina = 0
html_produtos = []
flag = 0

while flag == 0:
    items = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})
    if items != []:
        try:
            pagina = pagina + 1
            browser.get(f'https://www.kabum.com.br/hardware/placa-de-video-vga?pagina={pagina}')
            html = soup(browser.page_source, 'html.parser')
        except Exception: break
        html_produtos.append(items)
    else:
        break

### Tranformação da lista de listas em uma lista normal para percorer com mais facildiade os atributos do produto

In [5]:
flat_list = []
for sublist in html_produtos:
    for item in sublist:
        flat_list.append(item)

### Percorrendo a lista, capurando e salvando cada atributo no dicionario

In [6]:
now = datetime.now()
produtos = []

for container in flat_list:
    
    # Estreutura de para salvar
    marca = container.div.img['alt']
    nome_produto = container.find('a', {"class":"sc-fzoLsD gnrNhT item-nome"}).text

    if container.find('div', {"class":"sc-fzpjYC hKlXgM"}) != None:
        preco_antigo_cartao = (container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2]
    else: 
        preco_antigo_cartao = '0'
    preco_atual_cartao =  (container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1]
    preco_boleto = (container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1]
    
    image = container.find('div', {"class":"sc-fznKkj fEpBwY"})
    if "indisponivel" in image.img['src']:
        disponibilidade = 'Indisponivel'
    elif "disponivel" in image.img['src']:
        disponibilidade = 'Disponivel'

    produto = {
        "marca" : marca,
        "nome_produto": nome_produto,
        "preco_antigo_cartao": preco_antigo_cartao,
        "preco_atual_cartao": preco_atual_cartao,
        "preco_boleto": preco_boleto,
        "disponibilidade": disponibilidade,
        "data" : now.strftime("%Y-%m-%d"),
        "hora":  now.strftime("%H:%M:%S"),
        "loja": "Kabum" 
    }
    produtos.append(produto)


### Transformando o dict em Dataframe e visualizando uma amostra dos dados

In [7]:
df = pd.DataFrame(produtos)
df.sample(10)

,marca,nome_produto,preco_antigo_cartao,preco_atual_cartao,preco_boleto,disponibilidade,data,hora,loja
83,Asus,Placa de Vídeo Asus NVIDIA Dual GeForce RTX 20...,"5.882,24","5.411,65","4.599,90",Disponivel,2020-09-17,19:23:41,Kabum
174,Zotac,Placa de Vídeo Zotac Gaming NVIDIA GeForce GTX...,0,"1.529,29","1.299,90",Disponivel,2020-09-17,19:23:41,Kabum
12,Zotac,Placa de Vídeo Zotac NVIDIA GeForce GT 710 2GB...,"411,47","378,84","359,90",Disponivel,2020-09-17,19:23:41,Kabum
380,MSI,Placa de Vídeo MSI NVIDIA GeForce RTX 2070 Sup...,"5.411,65","4.235,18","3.599,90",Indisponivel,2020-09-17,19:23:41,Kabum
14,Evga,Placa de Vídeo EVGA NVIDIA GeForce RTX 2060 KO...,0,"2.823,41","2.399,90",Disponivel,2020-09-17,19:23:41,Kabum
93,Gigabyte,Placa de Vídeo Gigabyte NVIDIA GeForce GTX 165...,"1.949,89","1.882,24","1.599,90",Disponivel,2020-09-17,19:23:41,Kabum
66,Evga,Placa de Vídeo EVGA NVIDIA GeForce RTX 2080 Su...,"5.961,06","5.058,71","4.299,90",Disponivel,2020-09-17,19:23:41,Kabum
341,Galax,Placa de Vídeo Galax NVIDIA GeForce RTX 2080 T...,"11.114,00","9.411,65","7.999,90",Indisponivel,2020-09-17,19:23:41,Kabum
316,MSI,Placa de Vídeo MSI NVIDIA GeForce GTX 1050 Aer...,0,"752,82","639,90",Indisponivel,2020-09-17,19:23:41,Kabum
179,Galax,Placa de Vídeo Galax NVIDIA GeForce GTX 1660 S...,"2.599,88","2.352,82","1.999,90",Disponivel,2020-09-17,19:23:41,Kabum



### Criando a conexão com o banco e salvando os dados

In [8]:
engine = alchemy_connection('db_hardware')
df.to_sql('graphic_card', con=engine, if_exists='append', index = False)

----------------------------------------------------------------------------------------------------

In [19]:
import datetime
import sys
from concurrent.futures import ThreadPoolExecutor, wait
from time import sleep, time

In [ ]:
from scrapers.scraper import connect_to_base, get_driver, parse_html, write_to_file


def run_process(page_number, filename, headless):

    # init browser
    browser = get_driver(headless)

    if connect_to_base(browser, page_number):
        sleep(2)
        html = browser.page_source
        output_list = parse_html(html)
        write_to_file(output_list, filename)

        # exit
        browser.quit()
    else:
        print("Error connecting to hacker news")
        browser.quit()


if __name__ == "__main__":

    # headless mode?
    headless = False
    if len(sys.argv) > 1:
        if sys.argv[1] == "headless":
            print("Running in headless mode")
            headless = True

    # set variables
    start_time = time()
    output_timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    output_filename = f"output_{output_timestamp}.csv"
    futures = []

    # scrape and crawl
    with ThreadPoolExecutor() as executor:
        for number in range(1, 21):
            futures.append(
                executor.submit(run_process, number, output_filename, headless)
            )

    wait(futures)
    end_time = time()
    elapsed_time = end_time - start_time
    print(f"Elapsed run time: {elapsed_time} seconds")

In [9]:
browser2 = webdriver.Chrome(executable_path=r'chromedriver.exe', options=options)
#--| Parse or automation
browser2.get('https://www.pichau.com.br/hardware/placa-de-video?p=1')
web = soup(browser2.page_source, 'html.parser')

In [10]:
pagina = 0
html_produtos = []
flag = 0

while pagina != 5:
    items = web.findAll('div', {"class":"product-item-info"})
    if items != [] | item:
        try:
            pagina = pagina + 1
            browser.get(f'https://www.pichau.com.br/hardware/placa-de-video?p={pagina}')
            web = soup(browser.page_source, 'html.parser')
        except Exception: break
        html_produtos.append(items)
    else:
        break

In [11]:
flat_list = []
for sublist in html_produtos:
    for item in sublist:
        flat_list.append(item)

In [12]:
items[15].div

IndexError: list index out of range

In [13]:
# now = datetime.now()
# produtos = []

# for container in flat_list:
    
#     # Estreutura de para salvar
#     marca = container.div.img['alt']
#     nome_produto = container.find('a', {"class":"sc-fzoLsD gnrNhT item-nome"}).text

#     if container.find('div', {"class":"sc-fzpjYC hKlXgM"}) != None:
#         preco_antigo_cartao = (container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2]
#     else: 
#         preco_antigo_cartao = '0'
#     preco_atual_cartao =  (container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1]
#     preco_boleto = (container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1]
    
#     image = container.find('div', {"class":"sc-fznKkj fEpBwY"})
#     if "indisponivel" in image.img['src']:
#         disponibilidade = 'Indisponivel'
#     elif "disponivel" in image.img['src']:
#         disponibilidade = 'Disponivel'

#     produto = {
#         "marca" : marca,
#         "nome_produto": nome_produto,
#         "preco_antigo_cartao": preco_antigo_cartao,
#         "preco_atual_cartao": preco_atual_cartao,
#         "preco_boleto": preco_boleto,
#         "disponibilidade": disponibilidade,
#         "data" : now.strftime("%Y-%m-%d"),
#         "hora":  now.strftime("%H:%M:%S"),
#         "loja": "Kabum" 
#     }
#     produtos.append(produto)

In [14]:
# containers = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})
# https://www.kabum.com.br/cgi-local/site/listagem/listagem.cgi?string=processador&btnG=&pagina=1&ordem=5&limite=30&prime=false&marcas=[%225%22,%226%22]&tipo_produto=[]&filtro=[]

In [15]:
# images = html.findAll('div', {"class":"sc-fznKkj fEpBwY"})

# # images.img['src']
# for image in images:
#     print(image.img['src'])
#     # if "indisponivel" in image.img['src']:
#     #     print('Indisponivel')
#     # elif "disponivel" in image.img['src']:
#     #     print('Disponivel')

In [16]:
# preco_antigo_cartao = float((container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2].replace('.', '').replace(',', '.'))
# preco_atual_cartao =  float((container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1].replace('.', '').replace(',', '.'))
# preco_boleto = float((container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1].replace('.', '').replace(',', '.'))


In [17]:
# print(preco_antigo_cartao)
# # print(re.search('^(([1-9]\d{0,2}(\.\d{3})*)|(([1-9]\.\d*)?\d))(\,\d\d)?', preco_atual_cartao).group())
# print(preco_atual_cartao)
# print(preco_boleto)